We load an Excel file that contains information about the LOR keys used in the first data set. This is somewhat unneccessary as the first data set already contains columns with both the keys and corresponding name of the LOR. We do it anyways as an exercise.

The Excel file is extremely messy and contains a lot of cells that we don't need.

In [21]:
import pandas as pd
import geopandas as gpd
import sqlite3
import re
from pathlib import Path
import shapely.wkb

import warnings
warnings.filterwarnings('ignore')

lor_keys = pd.concat(pd.read_excel("data/LOR-Schluesselsystematik.xls", 
    sheet_name=[1,2,3,4,5,6,7,8,9,10,11,12], 
    header=3,
    usecols=[1,4,7,10,11],
    names=["BEZ", "PRG", "REG", "PLR", "name"]))

We generate the key which follows the format `<BEZ><PRG><REG><PLR>` (with zero padding) from the corresponding columns.

In [22]:
lor_keys.dropna(subset = ["PLR"], inplace=True)
lor_keys.fillna(method='ffill', inplace=True)
lor_keys.BEZ = [f'{int(n):02}' for n in lor_keys.BEZ] # parse as string (with zero padding)
lor_keys.PRG = [f'{int(n):02}' for n in lor_keys.PRG]
lor_keys.REG = [f'{int(n):02}' for n in lor_keys.REG]
lor_keys.PLR = [f'{int(n):02}' for n in lor_keys.PLR]
lor_keys["key"] = lor_keys.BEZ + lor_keys.PRG + lor_keys.REG + lor_keys.PLR # create key

However, the file that we use is not up to date: one PLR, added in 2019, is missing. The other data sets that we work with does include the new PLR, we therefore add it.

In [23]:
new_entry = {'BEZ': '09', 'PRG': '04', 'REG': '15', 'PLR': '03', 'name': 'Siedlung Kämmereiheide', 'key': '09041503'}
lor_keys.loc[len(lor_keys)] = new_entry

We now load a GeoPackage file that we have created in QGIS from the output file of the last notebook. It contains the number of accidents per PLR.

In [24]:
num_accidents = gpd.read_file("data/num_accidents.gpkg")
num_accidents.rename(columns={"PLANUNGSRAUM": "name", "FLAECHENGROESSE_IN_M2": "area_m2"}, inplace=True)

Unfortunately, PLRs are named inconsistently across data sets. For instance, sometimes "Straße" (street) is abbreviated, sometimes it isn't. We transform both name columns by replacing substrings using Regex, so that the names in both data sets are the same.

In [25]:
num_accidents["name"] = [re.sub(r"(-|\.|icher|iche|ische|ord|est|aße|\s)", "", s) for s in num_accidents["name"]]
lor_keys["name"] = [re.sub(r"(-|\.|icher|iche|ische|ord|est|aße|\s)", "", s) for s in lor_keys["name"]]
num_accidents["name"] = num_accidents["name"].str.lower().astype(str)
lor_keys["name"] = lor_keys["name"].str.lower().astype(str)

Two PLRs share the name "Schloßstraße". We need to rename one.

In [26]:
num_accidents.loc[(num_accidents.BEZIRKSNAME == "Charlottenburg-Wilmersdorf") & (num_accidents.name == "Schloßstraße"), ["name"]] = "schloßstr-charl"
lor_keys.loc[lor_keys.key == "04030417", "name"] = "schloßstr-charl"

Now we can join the DataFrames and create the response variable for our model, the daily average of accidents per m2 for each PLR.

In [27]:
num_accidents_joined = num_accidents.merge(lor_keys, on="name")

num_accidents_joined["mean_accidents_per_m2_2019"] = (num_accidents["num_accidents_2019"] / num_accidents["area_m2"]) / 365

num_accidents_joined = num_accidents_joined[[
    "name", 
    "key", 
    "area_m2", 
    "num_accidents_2019", 
    "mean_accidents_per_m2_2019",
]]

num_accidents_joined["key"] = num_accidents_joined["key"].astype(int)

We turn the DataFrame into a GeoDataFrame and compute the centroid for each geometry. We need this information for Kriging.

In [28]:
num_accidents_joined_full = gpd.GeoDataFrame(num_accidents_joined, geometry=num_accidents.geometry)
num_accidents_joined_full.crs = 'epsg:4326'
num_accidents_joined_full["centroid"] = num_accidents_joined_full["geometry"].centroid
num_accidents_joined_full["centroid_lon"] = num_accidents_joined_full["centroid"].x
num_accidents_joined_full["centroid_lat"] = num_accidents_joined_full["centroid"].y

C:\Users\Lenovo\AppData\Local\Temp/ipykernel_17688/99606362.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  num_accidents_joined_full["centroid"] = num_accidents_joined_full["geometry"].centroid


With that done, we can load another GeoPackage that contains information on the daily average traffic volume per road in 2019. We will use this data to create our predictor variable. To do so, we need to find the roads that our PLRs intersect with and combine the values for all roads in this district to get the (average) daily number of cars per PLR. We also calculate a measure relative to the area of the PLR.

In [29]:
traffic = gpd.read_file("data/traffic.gpkg")
indices = num_accidents_joined_full["geometry"].map(lambda x: traffic["geometry"].intersects(x)) # for each lor, check which roads it intersects with
num_accidents_joined_full["daily_traffic_cars"] = [traffic.loc[i, "dtvw_kfz"].sum() for i in indices] # for these roads, calculate the sum of daily traffic (cars)
num_accidents_joined_full["daily_traffic_lorries"] = [traffic.loc[i, "dtvw_lkw"].sum() for i in indices] # (lorries)
num_accidents_joined_full["daily_traffic_cars_per_m2"] = num_accidents_joined_full["daily_traffic_cars"] / num_accidents_joined_full["area_m2"]
num_accidents_joined_full["daily_traffic_lorries_per_m2"] = num_accidents_joined_full["daily_traffic_lorries"] / num_accidents_joined_full["area_m2"]

We can finally save a GeoJSON file to the disk that contains the geometries of all PLRs along with information on the daily average of accidents per m2 in 2019. These data will be displayed in our app.

In [30]:
num_accidents_joined_full[[
    "geometry", 
    "key",
    "num_accidents_2019",
    "mean_accidents_per_m2_2019",
    ]].to_file('data/accidents_per_lor.geojson', driver='GeoJSON') 

We also create a SQLite database that we can fetch data from.

In [31]:
num_accidents_joined_full["geometry"] = [shapely.wkb.dumps(x) for x in num_accidents_joined_full["geometry"]] # geometry to binary for db storage
num_accidents_joined_full["centroid"] = [shapely.wkb.dumps(x) for x in num_accidents_joined_full["centroid"]]

Path('data/accidents.db').touch(exist_ok=True)

conn = sqlite3.connect('data/accidents.db')
num_accidents_joined_full.to_sql('accidents_per_lor', conn, if_exists='replace')

C:\Users\Lenovo\miniconda3\envs\geo_env\lib\site-packages\geopandas\geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
